In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:
df_all = pd.concat([train_data, val_data], axis=0)
df_all = pd.concat([df_all, test_data], axis=0)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, KFold

# Load and prepare the dataset
data = df_all
data['combined_posts'] = data['selected_posts'].apply(lambda posts: ' '.join(posts))
data['has_autism'] = data['diseases'].apply(lambda diseases: 1 if 'autism' in diseases else 0)

# Define features and target labels
X = data['combined_posts']
y = data['has_autism']

# Define a pipeline for text processing and classification
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('classifier', RandomForestClassifier())
])

# Define custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    else:
        specificity = np.nan
    return specificity

specificity_scorer = make_scorer(specificity_score, greater_is_better=True)

# Perform 5-fold cross-validation and calculate metrics
kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = cross_val_score(pipeline, X, y, cv=kf)
sensitivity_scores = cross_val_score(pipeline, X, y, cv=kf)
specificity_scores = cross_val_score(pipeline, X, y, cv=kf, scoring=specificity_scorer)

# Calculate mean and standard deviation for each metric
f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
sensitivity_mean, sensitivity_std = np.mean(sensitivity_scores), np.std(sensitivity_scores)
specificity_mean, specificity_std = np.mean(specificity_scores), np.std(specificity_scores)

print("Metrics after 5-fold cross-validation:")
print(f"F1 Score: {f1_mean:.3f} ± {f1_std:.3f}")
print(f"Sensitivity: {sensitivity_mean:.3f} ± {sensitivity_std:.3f}")
print(f"Specificity: {specificity_mean:.3f} ± {specificity_std:.3f}")

Metrics after 5-fold cross-validation:
F1 Score: 0.973 ± 0.002
Sensitivity: 0.973 ± 0.002
Specificity: 1.000 ± 0.000


In [ ]:
len(y)

26605

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, KFold

# Load and prepare the dataset
data = df_all
data['combined_posts'] = data['selected_posts'].apply(lambda posts: ' '.join(posts))
data['has_autism'] = data['diseases'].apply(lambda diseases: 1 if 'autism' in diseases else 0)

# Define features and target labels
X = data['combined_posts']
y = data['has_autism']

# Define a pipeline for text processing and classification
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('classifier', RandomForestClassifier())
])

# Define custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    else:
        specificity = np.nan
    return specificity

specificity_scorer = make_scorer(specificity_score, greater_is_better=True)

# Perform 5-fold cross-validation and calculate metrics
kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = cross_val_score(pipeline, X, y, cv=kf)
sensitivity_scores = cross_val_score(pipeline, X, y, cv=kf)
specificity_scores = cross_val_score(pipeline, X, y, cv=kf, scoring=specificity_scorer)

# Calculate mean and standard deviation for each metric
f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
sensitivity_mean, sensitivity_std = np.mean(sensitivity_scores), np.std(sensitivity_scores)
specificity_mean, specificity_std = np.mean(specificity_scores), np.std(specificity_scores)

print("Metrics after 5-fold cross-validation:")
print(f"F1 Score: {f1_mean:.3f} ± {f1_std:.3f}")
print(f"Sensitivity: {sensitivity_mean:.3f} ± {sensitivity_std:.3f}")
print(f"Specificity: {specificity_mean:.3f} ± {specificity_std:.3f}")


Metrics after 5-fold cross-validation:
F1 Score: 0.883 ± 0.005
Sensitivity: 0.902 ± 0.003
Specificity: 0.987 ± 0.002


In [ ]:
autism_keywords = {
    'item_1': ['social', 'group activities', 'others'],
    'item_2': ['routine', 'same way', 'repetition'],
    'item_3': ['imagine', 'picture in mind', 'visualization'],
    'item_4': ['absorbed', 'focus', 'lose sight'],
    'item_5': ['small sounds', 'notices sounds'],
    'item_6': ['car number plates', 'strings of information'],
    'item_7': ['impolite', 'polite'],
    'item_8': ['reading a story', 'imagine characters'],
    'item_9': ['fascinated by dates'],
    'item_10': ['social group', 'track of conversations'],
    'item_11': ['social situations', 'easy'],
    'item_12': ['notice details'],
    'item_13': ['library', 'party'],
    'item_14': ['making up stories', 'creative writing'],
    'item_15': ['drawn to people', 'interested in people'],
    'item_16': ['strong interests', 'upset if cannot pursue'],
    'item_17': ['social chit-chat', 'small talk'],
    'item_18': ['talks', 'get a word in edgeways'],
    'item_19': ['fascinated by numbers'],
    'item_20': ['reading a story', 'work out intentions'],
    'item_21': ['enjoy reading fiction'],
    'item_22': ['hard to make new friends'],
    'item_23': ['notices patterns'],
    'item_24': ['theatre', 'museum'],
    'item_25': ['daily routine', 'disturbed'],
    'item_26': ['keep a conversation going'],
    'item_27': ['read between the lines'],
    'item_28': ['whole picture', 'small details'],
    'item_29': ['remembering phone numbers'],
    'item_30': ['notice small changes', 'appearance'],
    'item_31': ['tell if someone is bored'],
    'item_32': ['do more than one thing', 'multitask'],
    'item_33': ['talk on the phone', 'turn to speak'],
    'item_34': ['doing things spontaneously'],
    'item_35': ['last to understand the joke'],
    'item_36': ['work out what someone is thinking', 'looking at their face'],
    'item_37': ['switch back quickly', 'interruption'],
    'item_38': ['good at social chit-chat'],
    'item_39': ['going on and on about the same thing'],
    'item_40': ['playing games', 'pretending'],
    'item_41': ['collect information', 'categories of things'],
    'item_42': ['imagine being someone else'],
    'item_43': ['plan activities carefully'],
    'item_44': ['enjoys social occasions'],
    'item_45': ['work out people’s intentions'],
    'item_46': ['new situations', 'anxious'],
    'item_47': ['enjoys meeting new people'],
    'item_48': ['good diplomat'],
    'item_49': ['remembering dates of birth'],
    'item_50': ['play games with children', 'pretending'],
}


In [ ]:

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, confusion_matrix
from sklearn.utils import resample
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load and prepare the training data
train_data['combined_posts'] = train_data['selected_posts'].apply(lambda posts: ' '.join(posts))

# Advanced text preprocessing
def preprocess_text(text):
    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_words = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

train_data['processed_posts'] = train_data['combined_posts'].apply(preprocess_text)

# Adjust the scoring function to be more sensitive
def score_autism(posts, keywords):
    scores = []
    for post in posts:
        score = 0
        for key, value in keywords.items():
            if any(word in post for word in value):
                score += 1
        scores.append(score)
    return scores


train_data['autism_score'] = score_autism(train_data['processed_posts'], autism_keywords)
train_data['has_autism'] = train_data['diseases'].apply(lambda x: 1 if 'autism' in x else 0)
train_data['post_length'] = train_data['combined_posts'].apply(len)

# Load and prepare the test data
test_data['combined_posts'] = test_data['selected_posts'].apply(lambda posts: ' '.join(posts))
test_data['processed_posts'] = test_data['combined_posts'].apply(preprocess_text)
test_data['autism_score'] = score_autism(test_data['processed_posts'], autism_keywords)
test_data['has_autism'] = test_data['diseases'].apply(lambda x: 1 if 'autism' in x else 0)
test_data['post_length'] = test_data['combined_posts'].apply(len)

# Define features and target labels for training
X_train = train_data[['processed_posts', 'post_length', 'autism_score']]
y_train = train_data['has_autism']

# Define features for testing
X_test = test_data[['processed_posts', 'post_length', 'autism_score']]
y_test = test_data['has_autism']

# Preprocessing for numerical and text data
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=1000), 'processed_posts'),
        ('num', StandardScaler(), ['post_length', 'autism_score'])
    ]
)

# Define and train the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=10))
])

# Train the model on the training data
pipeline.fit(X_train, y_train)

# Function to perform bootstrapped sampling and calculate metrics on the test set
def bootstrap_metrics(X, y, model, n_iterations=100, sample_size=None):
    f1_scores = []
    sensitivities = []  # Recall
    specificities = []

    for _ in range(n_iterations):
        X_sample, y_sample = resample(X, y, n_samples=sample_size)
        y_pred = model.predict(X_sample)

        f1_scores.append(f1_score(y_sample, y_pred, average='weighted'))
        sensitivities.append(recall_score(y_sample, y_pred, average='weighted'))

        # Calculate specificity using binary assumption; adjust as needed for multi-class
        cm = confusion_matrix(y_sample, y_pred, labels=np.unique(y_sample))
        if cm.shape == (2, 2):  # binary case
            tn, fp, fn, tp = cm.ravel()
            specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
        else:  # multi-class or other issues
            specificity = np.nan  # or handle differently
        specificities.append(specificity)

    return {
        'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
        'sensitivity': (np.mean(sensitivities), np.std(sensitivities)),
        'specificity': (np.mean(specificities), np.std(specificities))
    }

# Perform bootstrapped evaluation on the test set
metrics = bootstrap_metrics(X_test, y_test, pipeline, n_iterations=100)
print("Metrics after bootstrap sampling on test set for autism detection:")
print(f"F1 Score: {metrics['f1_score'][0]:.3f} ± {metrics['f1_score'][1]:.3f}")
print(f"Sensitivity: {metrics['sensitivity'][0]:.3f} ± {metrics['sensitivity'][1]:.3f}")
print(f"Specificity: {metrics['specificity'][0]:.3f} ± {metrics['specificity'][1]:.3f}")


Metrics after bootstrap sampling on test set for autism detection:
F1 Score: 0.959 ± 0.004
Sensitivity: 0.972 ± 0.003
Specificity: 1.000 ± 0.000


In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load and prepare the data
data = df_all
data['combined_posts'] = data['selected_posts'].apply(lambda posts: ' '.join(posts))

# Advanced text preprocessing
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_words = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

data['processed_posts'] = data['combined_posts'].apply(preprocess_text)

# Adjust the scoring function to be more sensitive
def score_autism(posts, keywords):
    scores = []
    for post in posts:
        score = 0
        for key, value in keywords.items():
            if any(word in post for word in value):
                score += 1
        scores.append(score)
    return scores


data['autism_score'] = score_autism(data['processed_posts'], autism_keywords)
data['has_autism'] = data['diseases'].apply(lambda x: 1 if 'autism' in x else 0)
data['post_length'] = data['combined_posts'].apply(len)

# Define features and target labels
X = data[['processed_posts', 'post_length', 'autism_score']]
y = data['has_autism']

# Preprocessing for numerical and text data
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=1000), 'processed_posts'),
        ('num', StandardScaler(), ['post_length', 'autism_score'])
    ]
)

# Define and train the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=10))
])

# Define custom scorer for specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    else:
        specificity = np.nan
    return specificity

specificity_scorer = make_scorer(specificity_score, greater_is_better=True)

# Perform 5-fold cross-validation and calculate metrics
kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='f1_weighted')
sensitivity_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='recall_weighted')
specificity_scores = cross_val_score(pipeline, X, y, cv=kf, scoring=specificity_scorer)

# Calculate mean and standard deviation for each metric
f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
sensitivity_mean, sensitivity_std = np.mean(sensitivity_scores), np.std(sensitivity_scores)
specificity_mean, specificity_std = np.mean(specificity_scores), np.std(specificity_scores)

print("Metrics after 5-fold cross-validation for autism detection:")
print(f"F1 Score: {f1_mean:.3f} ± {f1_std:.3f}")
print(f"Sensitivity: {sensitivity_mean:.3f} ± {sensitivity_std:.3f}")
print(f"Specificity: {specificity_mean:.3f} ± {specificity_std:.3f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Metrics after 5-fold cross-validation for autism detection:
F1 Score: 0.960 ± 0.003
Sensitivity: 0.973 ± 0.002
Specificity: 1.000 ± 0.000
